In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(action_space.low)
print(action_space.high)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1. -1. -1. -1.]
[1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-06-25 11:08:35,687	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-25_11-08-35_1449/logs.
2019-06-25 11:08:35,803	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:57075 to respond...
2019-06-25 11:08:35,918	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:34041 to respond...
2019-06-25 11:08:35,921	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-06-25 11:08:35,965	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-25_11-08-35_1449/logs.
2019-06-25 11:08:35,968	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:57075',
 'object_store_address': '/tmp/ray/session_2019-06-25_11-08-35_1449/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-06-25_11-08-35_1449/sockets/raylet',
 'webui_url': None}

In [5]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks
import games_configurations

a2c_config = games_configurations.bipedalwalker_config
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
#a2c_config = games_configurations.quadrupped_config
#a2c_config = a2c_games_configurations.mountain_car_cont_config
agent = A2CAgent(sess,'walker', obs_space, False, action_space, a2c_config)
#agent.restore('nn/latest_q')
agent.train()
#agent.save('nn/latest')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


2019-06-25 11:08:37,321	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=1540) 
(pid=1540) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=1540) For more information, please see:
(pid=1540)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=1540)   * https://github.com/tensorflow/addons
(pid=1540) If you depend on functionality not listed there, please file an issue.
(pid=1540) 
(pid=1537) 
(pid=1537) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=1537) For more information, please see:
(pid=1537)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=1537)   * https://github.com/tensorflow/addons
(pid=1537) If you depend on functionality not listed there, please file an issue.
(pid=1537) 
(pid=1539) 
(pid=1539) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=1539) For more information, please see:
(pid=1539)   * https://github.com/tensorflow/community/blob/master/rf

(pid=1541) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=1541) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=1585) 
(pid=1585) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=1585) For more information, please see:
(pid=1585)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=1585)   * https://github.com/tensorflow/addons
(pid=1585) If you depend on functionality not listed there, please file an issue.
(pid=1585) 
(pid=1583) 
(pid=1583) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=1583) For more information, please see:
(pid=1583)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=1583)   * https://github.com/tensorflow/addons
(pid=1583) If you depend on functionality not listed there, please file an issue.
(pid=158

Frames per seconds:  2162.6802615276897
saving next best rewards:  -103.37809646363101
Frames per seconds:  2202.6224463552217
Frames per seconds:  2158.9560706901384
Frames per seconds:  2195.9917328845877
Frames per seconds:  2164.2809026203495
Frames per seconds:  2160.1820079107524
Frames per seconds:  2182.0328653738975
Frames per seconds:  2218.9459105212936
Frames per seconds:  2121.8385765047196
Frames per seconds:  2165.279539212906
Frames per seconds:  2171.0520462510904
Frames per seconds:  2156.1400111096023
Frames per seconds:  2175.210512601503
Frames per seconds:  2206.5953020740226
Frames per seconds:  2180.545625916553
Frames per seconds:  2206.5241667610676
Frames per seconds:  2223.1349863421087
Frames per seconds:  2165.261527773787
saving next best rewards:  -99.97928904494215
Frames per seconds:  2121.1857177675474
saving next best rewards:  -96.7006454571431
Frames per seconds:  2171.990488176911
saving next best rewards:  -96.07845322828763
Frames per seconds:  

...

Frames per seconds:  2258.975971833853
Frames per seconds:  2588.465658573657
Frames per seconds:  2624.746565497551
Frames per seconds:  2601.2412442506907
Frames per seconds:  2660.7928237964043
Frames per seconds:  2541.416556730086
Frames per seconds:  2620.8697240865254
Frames per seconds:  2686.384879322075
Frames per seconds:  2571.41865618853
Frames per seconds:  2679.2138184857804
Frames per seconds:  2577.825904496675
Frames per seconds:  2612.223319121088
Frames per seconds:  2542.4832027599914
Frames per seconds:  2648.8030701680677
Frames per seconds:  2591.6242206777283
Frames per seconds:  2620.970084251557
Frames per seconds:  2616.457089700843
Frames per seconds:  2646.841373496196
Frames per seconds:  2625.319332338618
Frames per seconds:  2148.413642555521
Frames per seconds:  2666.6910650578234
Frames per seconds:  2586.6374592221327
Frames per seconds:  2675.001250938515
Frames per seconds:  2600.705311806959
Frames per seconds:  2646.313391510568
Frames per second

Frames per seconds:  2535.109121046775
Frames per seconds:  2639.2955982881626
Frames per seconds:  2551.4082505567326
Frames per seconds:  2641.9817142101406
Frames per seconds:  2504.027051860282
Frames per seconds:  2593.8917737976285
Frames per seconds:  2573.1691281458466
Frames per seconds:  2669.6838131704367
Frames per seconds:  2649.905784797631
Frames per seconds:  2532.271478667661
Frames per seconds:  2672.118345569748
Frames per seconds:  2503.8493236713357
Frames per seconds:  2511.083519449206
Frames per seconds:  2507.5195777760264
Frames per seconds:  2633.4011284470344
Frames per seconds:  2552.8781444625006
Frames per seconds:  2614.340479505347
Frames per seconds:  2096.873253055105
Frames per seconds:  2259.59991150929
Frames per seconds:  2486.9249697129894
Frames per seconds:  2541.2579150876136
Frames per seconds:  2692.328901665621
Frames per seconds:  2566.86052996063
Frames per seconds:  2635.8839490322334
Frames per seconds:  2548.12616138631
Frames per seco

Frames per seconds:  2629.8694184882756
Frames per seconds:  2572.1331925985482
Frames per seconds:  2587.1146230012205
Frames per seconds:  2538.8131304040703
Frames per seconds:  2644.6895882026856
Frames per seconds:  2575.638083812621
Frames per seconds:  2633.976871421854
Frames per seconds:  2584.6520124708154
Frames per seconds:  2631.0805237758527
Frames per seconds:  2591.3791165230487
Frames per seconds:  2605.4003451044337
Frames per seconds:  2617.026244037575
Frames per seconds:  2574.324704235111
Frames per seconds:  2640.039435543592
Frames per seconds:  2569.6009974555072
Frames per seconds:  2639.8102370715155
Frames per seconds:  2174.8222511075473
Frames per seconds:  2598.886128891816
Frames per seconds:  2580.962803205346
Frames per seconds:  2630.6877086535583
Frames per seconds:  2595.6051422985634
Frames per seconds:  2516.737484116453
Frames per seconds:  2659.400244022728
Frames per seconds:  2399.7800209109314
Frames per seconds:  2284.309807954299
Frames per

Exception in thread ray_import_thread:
Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/ray/import_thread.py", line 70, in _run
    msg = import_pubsub_client.get_message()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/client.py", line 3135, in get_message
    response = self.parse_response(block=False, timeout=timeout)
  File "/home/trrrrr/anaconda3/envs/

KeyboardInterrupt: 

In [6]:
ray.shutdown()

In [7]:
import numpy as np
#agent.save('nn/latest_q')
#agent.restore('nn/carRoboschoolAnt-v1')
#agent.restore('nn/rob_20QuadruppedWalk-v1')
agent.restore('nn/walkerBipedalWalker-v2')
env = env_configurations.configurations[env_name]['ENV_CREATOR']()
def evaluate(env,t_max=5000):
    rewards = []
    #env._max_episode_steps = 5000
    env.batch_mode = True
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(5000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env.close()




Instructions for updating:
Use standard file APIs to check for files with this prefix.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from nn/walkerBipedalWalker-v2


INFO:tensorflow:Restoring parameters from nn/walkerBipedalWalker-v2


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
reset
[[-0.9884704  -0.9975635   0.01064356 -0.98837036]]
[[-0.9379955  -0.99737126  0.52504635 -0.92124254]]
[[-0.79891247 -1.          0.8729633  -0.82696694]]
[[-0.64705765 -0.94529206 -0.31894928 -0.87715566]]
[[-0.4060015  -0.62475514  1.         -1.        ]]
[[-0.4442265 -0.740653   0.9395771 -0.7275956]]
[[ 0.02031359 -1.          0.9884893  -1.        ]]
[[-0.44451937 -1.          1.         -0.44298398]]
[[ 0.5786416  -1.         -0.14228785  0.7054284 ]]
[[ 0.73159206 -0.9579528   1.          0.06323871]]
[[ 0.1616136  -0.6020779   0.23635364  0.19134825]]
[[ 0.7731517   0.17848855 -0.18052608 -0.32754236]]
[[ 0.02662207 -0.57089704 -1.          0.1062468 ]]
[[0.8009611  0.5273256  0.38122448 0.8292704 ]]
[[0.11923042 0.9434695  0.379886   0.9489231 ]]
[[-0.05873461  0.98841137  0

...

[[ 0.88080966 -0.960393    0.1587519   0.37007728]]
[[ 0.5251874  -0.9605047   0.48999587  0.2531663 ]]
[[ 0.40568763 -0.04425987  0.42651242  0.3009866 ]]
[[-0.1601992   0.60443795  0.38670224  0.49448755]]
[[-0.7255744   0.96364456  0.26025116  0.60042554]]
[[-0.9140203   0.97759736 -0.02540325 -0.221008  ]]
[[-0.81420493  0.340593    0.327781   -0.56248784]]
[[ 0.43643892 -0.8873087   0.33403403 -0.09418894]]
[[ 0.60164255 -0.957981    0.43982258  0.25520396]]
[[ 0.5353548  -0.952404    0.16357277  0.09655868]]
[[-0.06430838 -0.8656422   0.35085154 -0.34375218]]
[[-0.15619065 -0.8182886   0.03488034  0.05527185]]
[[ 0.19906753 -0.8931233   0.27030447  0.4719852 ]]
[[ 0.5390905 -0.8175511  0.6607709  0.2682858]]
[[ 0.17129794 -0.5294893   0.435138    0.40347004]]
[[0.0764499  0.47478086 0.2813011  0.52554566]]
[[ 0.03005386  0.45472237 -0.04127776  0.20079044]]
[[0.06506003 0.96158534 0.20722252 0.02355372]]
[[-0.09348301  0.95042497 -0.04217194  0.10690025]]
[[ 0.5713469   0.9742521

[[-0.9513398   0.98436666  0.22837114  0.17414732]]
[[-0.1044338   0.9151123  -0.33771515  0.25606737]]
[[-0.85465735 -0.10972455  0.33184662 -0.7397227 ]]
[[-0.23149265 -0.75117743  0.4547129  -0.4231885 ]]
[[ 0.89291984 -0.99078304  0.14923435 -0.4515108 ]]
[[ 0.9681345  -0.95034486  0.6827436   0.21846825]]
[[ 0.783027   -1.          0.28664193  0.74844766]]
[[ 0.6113569  -0.9648501   0.18768752  0.44623944]]
[[ 0.0130861   0.13063338 -0.21090528  0.27175054]]
[[-0.6761224   0.79639095  0.4145646   0.5509894 ]]
[[-0.72549915  0.9699639   0.6364086   0.1519237 ]]
[[-0.70891845  0.5230792   0.36345178 -0.94325095]]
[[ 0.35403916 -0.7777057   0.70409274 -0.02301225]]
[[ 0.709822   -0.93663377  0.54542124  0.21514031]]
[[ 0.7336252 -0.9174462  0.3755533  0.270471 ]]
[[ 0.6145316  -0.81586343  0.30446944  0.0207144 ]]
[[ 0.04240812 -0.04081618  0.05610783  0.27093086]]
[[0.19700593 0.6523596  0.190212   0.81190324]]
[[0.39028573 0.9060285  0.8836738  0.6790299 ]]
[[0.33411422 0.9899265  

[[0.33188656 0.9142397  0.1330444  0.4342632 ]]
[[-0.37977558  0.9361429   0.58648455 -0.194002  ]]
[[-0.39655113  0.9434553   0.9171367  -0.5580238 ]]
[[ 0.5236666   1.         -0.9287777  -0.24123845]]
[[ 0.7875368  -0.70469904  0.19338477 -0.6488245 ]]
[[ 0.45858294 -0.90939754  0.46691036  0.06243956]]
[[ 0.81466866 -0.92072093  0.93388855 -0.2016804 ]]
[[ 0.7604205  -1.          0.53729886  0.18533748]]
[[ 0.7210548  -1.          0.31367552  0.47088513]]
[[0.598166   0.38226962 0.29564762 0.7524691 ]]
[[ 0.6695987   0.9527874  -0.11469224  0.56867945]]
[[0.38516685 1.         0.19571003 1.        ]]
[[-0.84329623  0.99081546  0.52952164  0.8942641 ]]
[[-0.931022    0.98279303  0.05470613  0.4148049 ]]
[[-0.9167409   0.9683342   0.23886552 -0.6935792 ]]
[[-0.7202523   0.8718245   0.26470357 -0.3945131 ]]
[[ 0.34587836 -0.7065433  -0.27359596 -0.7462666 ]]
[[ 0.8234731  -0.9342834   0.33157635 -0.28603986]]
[[ 0.6851132  -0.99028975 -0.22488551 -0.04427966]]
[[ 0.6099663  -0.9602952

[[ 0.43403727 -0.34602877 -0.01669516 -0.06979412]]
[[-0.09368367 -0.00456248  0.26645976 -0.500983  ]]
[[-0.16404097 -0.27619773 -0.14665848 -0.19558755]]
[[ 0.46327013 -0.5846287   0.5653893   0.3973456 ]]
[[ 0.5544343  -0.54837537  0.5843248   0.31228524]]
[[ 0.43901557 -0.06833612  0.47273707  0.36858168]]
[[0.29282635 0.40413326 0.33953667 0.25267234]]
[[-0.12336516  0.8621401  -0.0076964   0.05197297]]
[[ 0.09502362  0.9739518  -0.13243541 -0.18163042]]
[[ 0.13871065  0.8390645  -0.24665807 -0.10496417]]
[[ 0.38722652  0.66976863 -0.31772807  0.33869854]]
[[0.72798055 0.91148627 0.5413665  0.57165444]]
[[0.3628804  0.694918   0.35155728 0.4444495 ]]
[[-0.4721151   0.9442497  -0.20997512  0.03145129]]
[[ 0.0593424   0.94546837 -0.595281   -0.6703305 ]]
[[ 0.10705052  0.7730207  -0.6928956  -0.818114  ]]
[[ 0.80379915  0.4027913  -0.27040976 -0.93884784]]
[[ 0.38433182 -0.8999157  -0.67744434 -1.        ]]
[[ 0.89127356 -0.9102868  -0.40615508 -0.9463668 ]]
[[ 0.5449532  -0.9176889

[[ 0.1843247  -0.26488402  0.2873339  -0.18252602]]
[[0.01278748 0.59807354 0.27784386 0.51437604]]
[[0.15866897 0.9489364  0.23965287 0.4142648 ]]
[[0.2062284  0.9437288  0.6507368  0.28759727]]
[[0.32714087 1.         0.3158695  0.0563046 ]]
[[ 0.48795876  0.9711351   0.16525467 -0.02311149]]
[[ 0.36175728  0.9684122  -0.35551482 -0.44875497]]
[[ 0.21897921  0.89114654 -0.58201146 -0.85448885]]
[[ 0.56531274  0.61397225  0.41192478 -0.68818516]]
[[ 0.6471344  0.4911742  0.0253026 -1.       ]]
[[ 0.45520952 -0.6747352  -0.22230394 -0.9370519 ]]
[[ 0.9792223  -0.87067556 -0.74908    -0.97911865]]
[[ 0.8701782  -1.          1.         -0.87496066]]
[[ 0.7044087 -1.         1.        -0.2381768]]
[[ 0.7160443  -0.9941768   0.8810639   0.43351722]]
[[ 0.77963203 -0.95259935  0.4047929   0.58503056]]
[[ 0.8225605  -0.6309683   0.30181757  0.41756037]]
[[ 0.77004313 -0.22185043  0.09230666  0.5711508 ]]
[[0.5586482  0.71472496 0.09966724 0.77376306]]
[[ 0.28782707  0.8869174  -0.07125077  0

In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()